# Introduction

This notebook is intended to show how to use [Nimble's remote MCP server](https://docs.nimbleway.com/ai-agents/mcp-server) with Databricks model serving via popular agent-authoring libraries (LangChain, LlamaIndex).

By the end of this notebook, you will understand how to build and run simple agents that can use the tools accessible via Nimble's remote MCP server using LangChain and LlamaIndex.

For more details on getting started using these frameworks with Databricks Model Serving, see the [Databricks Agent Library Integrations](databricks_agent_library_integrations.ipynb) notebook.

## Using the Nimble MCP Server with a LangGraph Agent

In this section, we'll show how to build an agent using LangChain/LangGraph that can access both Databricks-hosted LLMs and Nimble's external tools via the MCP server.

We'll walk through:
- Installing the required libraries
- Authenticating with Databricks and Nimble
- Creating an agent that can use both LLM and external tools
- Running the agent on a real-world question

### Setup

In [0]:
%pip install langchain-mcp-adapters mcp databricks-langchain langgraph langchain mlflow python-dotenv
%restart_python

Now that we have installed the required packages, let's configure access to Databricks models and to the Nimble MCP server.

In this example, we use a `.env` file and the `python-dotenv` package to load our Nimble API key (which you can obtain from your [Nimble account settings](https://app.nimbleway.com/account_settings/api_keys)). The `.env` file just has one line:

```
NIMBLE_API_KEY=<your nimble API key>
```

In the code block below, we also set up the `llm` with `ChatDatabricks` and the MCP client with LangChain's `MultiServerMCPClient` to access the Nimble MCP server. Lastly, we get the list of tools to pass to the agent using `client.get_tools()` and initialize the agent with `create_react_agent`, passing in the LLM and the list of tools.

In [0]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from databricks_langchain import ChatDatabricks
from databricks.sdk import WorkspaceClient
import mlflow

import os

from dotenv import load_dotenv

mlflow.langchain.autolog()

load_dotenv()

w = WorkspaceClient()

os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")


client = MultiServerMCPClient({
    "nimble": {
        "url": "https://mcp.nimbleway.com/sse",
        "transport": "sse",
        "headers": {
            "Authorization": f"Bearer {os.environ['NIMBLE_API_KEY']}"
        }
    }
})

tools = await client.get_tools()
agent = create_react_agent(llm, tools)

### Running the agent

We can call on the agent with `await agent.ainvoke()`, passing in a list of messages. Note that, to use the tools from the MCP server, we need to use the asynchronous method `ainvoke`; this will not work with the synchronous `invoke` method.

In [0]:
await agent.ainvoke({"messages": [{"role": "user", "content": "What is the best Ethiopian restaurant on the northside of Chicago?"}]})

We can use Nimble's web extraction tool to teach us a little more about the tools available through Nimble's MCP server.

In [0]:
await agent.ainvoke({"messages": [{"role": "user", "content": "Create a table of the tools detailed here https://docs.nimbleway.com/ai-agents/mcp-server/available-tools and their descriptions using your web extraction tool"}]})

## Using the Nimble MCP Server with a LlamaIndex agent

In this section, we'll work through essentially the same process as above, but using LlamaIndex instead of LangChain. As before, we will obtain the tools from the MCP server and use them to build an agent. Our Nimble API key will, again, be loaded from a `.env` file.


### Setup


In [0]:
%pip install -U -qqq llama-index llama-index-llms-databricks mlflow python-dotenv llama-index-tools-mcp
%restart_python

In [0]:
from llama_index.llms.databricks import Databricks
from databricks.sdk import WorkspaceClient
import mlflow

w = WorkspaceClient()

tmp_token = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = Databricks(
    model="databricks-llama-4-maverick",
    api_key=tmp_token,
    api_base=f"{w.config.host}/serving-endpoints/"
)

We will use the LlamaIndex `BasicMCPClient` to get the list of tools from the MCP server and create a `McpToolSpec` object. We can then convert this to a list of tools using the `to_tool_list_async` method.

In [0]:
from dotenv import load_dotenv
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec
import os

load_dotenv()

async def setup_nimble_tools():
    mcp_client = BasicMCPClient(
        "https://mcp.nimbleway.com/sse",
        headers={"Authorization": f"Bearer {os.environ['NIMBLE_API_KEY']}"}
    )
    
    mcp_tool_spec = McpToolSpec(
        client=mcp_client,
        # Optional: filter to specific tools
        # allowed_tools=["nimble_web_search", "nimble_google_maps_search"]
    )
    
    tools = await mcp_tool_spec.to_tool_list_async()
    
    print(f"Loaded {len(tools)} Nimble tools:")
    for tool in tools:
        print(f"- {tool.metadata.name}: {tool.metadata.description}")
    
    return tools

# Get the tools
nimble_tools = await setup_nimble_tools()
nimble_tools

We will use the LlamaIndex `ReActAgent` to build our agent. We pass in the list of tools, the LLM, and a system prompt.

In [0]:
from llama_index.core.agent.workflow import ReActAgent

agent = ReActAgent(
    tools=nimble_tools,
    llm=llm,
    system_prompt="""You are a helpful research assistant with access to web scraping and data collection tools. 
    Always explain your answer in the final output. Tell the user which tools you used and how you found the information.""",
)

### Running the agent

We can run the agent with `await agent.run()`, passing in a question.


In [0]:
mlflow.llama_index.autolog()
await agent.run("What is the best Ethiopian restaurant in Uptown Chicago?")